In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"f1atnote","key":"b31c6a1b0663c414dff524a4d12a131b"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!pip install kaggle
!kaggle datasets download -d dasmehdixtr/drone-dataset-uav

Dataset URL: https://www.kaggle.com/datasets/dasmehdixtr/drone-dataset-uav
License(s): copyright-authors
 98% 703M/716M [00:04<00:00, 160MB/s]
100% 716M/716M [00:04<00:00, 175MB/s]


In [4]:
!unzip drone-dataset-uav.zip -d ./drone_dataset

Archive:  drone-dataset-uav.zip
  inflating: ./drone_dataset/dataset_xml_format/dataset_xml_format/foto00088.png  
  inflating: ./drone_dataset/dataset_xml_format/dataset_xml_format/foto00088.xml  
  inflating: ./drone_dataset/dataset_xml_format/dataset_xml_format/foto00117.png  
  inflating: ./drone_dataset/dataset_xml_format/dataset_xml_format/foto00117.xml  
  inflating: ./drone_dataset/dataset_xml_format/dataset_xml_format/foto00204.png  
  inflating: ./drone_dataset/dataset_xml_format/dataset_xml_format/foto00204.xml  
  inflating: ./drone_dataset/dataset_xml_format/dataset_xml_format/foto00233.png  
  inflating: ./drone_dataset/dataset_xml_format/dataset_xml_format/foto00233.xml  
  inflating: ./drone_dataset/dataset_xml_format/dataset_xml_format/foto00262.png  
  inflating: ./drone_dataset/dataset_xml_format/dataset_xml_format/foto00262.xml  
  inflating: ./drone_dataset/dataset_xml_format/dataset_xml_format/foto00291.png  
  inflating: ./drone_dataset/dataset_xml_format/dataset

In [5]:
!pip install tensorflow tensorflow-object-detection-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 804.0/804.0 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.9 MB/s eta 0:00:00
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detec

In [6]:
# label_map.pbtxt 생성
label_map_text = """
item {
  id: 1
  name: 'drone'
}
"""

with open("label_map.pbtxt", "w") as f:
    f.write(label_map_text)

In [8]:
import os

DATA_DIR = '/content/drone_dataset/dataset_xml_format/dataset_xml_format'

images = sorted([f for f in os.listdir(DATA_DIR) if f.endswith(('.png', '.jpg', '.jpeg'))])
xmls = sorted([f for f in os.listdir(DATA_DIR) if f.endswith('.xml')])

print(f'📷 이미지 수: {len(images)}')
print(f'📄 XML 수: {len(xmls)}')

print('\n🔎 예시 이미지 파일 5개:')
print(images[:5])


📷 이미지 수: 1087
📄 XML 수: 1097

🔎 예시 이미지 파일 5개:
['foto00088.png', 'foto00117.png', 'foto00204.png', 'foto00233.png', 'foto00262.png']


In [9]:
import xml.etree.ElementTree as ET

mismatches = []

for xml_file in xmls:
    tree = ET.parse(os.path.join(DATA_DIR, xml_file))
    root = tree.getroot()
    filename = root.find('filename').text
    if filename not in images:
        mismatches.append(filename)

print(f' 매칭 안 되는 XML 내 이미지 파일 수: {len(mismatches)}')
if mismatches:
    print('예시:', mismatches[:5])

 매칭 안 되는 XML 내 이미지 파일 수: 140
예시: ['pic_029.jpeg', 'pic_038.png', 'pic_042.png', 'pic_047.JPG', 'pic_067.png']


In [10]:
import os
import tensorflow as tf
import xml.etree.ElementTree as ET
from object_detection.utils import dataset_util

# 경로 수정
DATA_DIR = '/content/drone_dataset/dataset_xml_format/dataset_xml_format'
IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png']

def create_tf_example(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    filename = root.find('filename').text
    image_path = os.path.join(DATA_DIR, filename)

    # 파일 존재 여부 확인
    if not os.path.exists(image_path):
        print(f'[⚠️] 이미지 없음: {filename}, 스킵합니다.')
        return None

    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_image = fid.read()

    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    # 기본 값
    image_format = filename.split('.')[-1].encode('utf8')

    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for obj in root.findall('object'):
        name = obj.find('name').text
        bbox = obj.find('bndbox')
        xmins.append(float(bbox.find('xmin').text) / width)
        xmaxs.append(float(bbox.find('xmax').text) / width)
        ymins.append(float(bbox.find('ymin').text) / height)
        ymaxs.append(float(bbox.find('ymax').text) / height)
        classes_text.append(name.encode('utf8'))
        classes.append(1)  # 'drone' = 1

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_image),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


In [11]:
import os
import glob
import tensorflow as tf
import xml.etree.ElementTree as ET
from object_detection.utils import dataset_util

# 수정해줘야 하는 경로
LABEL_MAP = {'drone': 1}
ANNOTATION_DIR = '/content/drone_dataset/dataset_xml_format/dataset_xml_format'
OUTPUT_PATH = '/content/train.record'  # 또는 val.record 등으로 변경 가능

def create_tf_example(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    filename = root.find('filename').text
    image_path = os.path.join(ANNOTATION_DIR, filename)

    if not os.path.exists(image_path):
        print(f'[⚠️] 이미지 없음: {filename}, 스킵')
        return None

    with tf.io.gfile.GFile(image_path, 'rb') as fid:
        encoded_image = fid.read()

    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    image_format = filename.split('.')[-1].encode('utf8')

    xmins, xmaxs, ymins, ymaxs = [], [], [], []
    classes_text, classes = [], []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        if class_name not in LABEL_MAP:
            continue

        bndbox = obj.find('bndbox')
        xmins.append(float(bndbox.find('xmin').text) / width)
        xmaxs.append(float(bndbox.find('xmax').text) / width)
        ymins.append(float(bndbox.find('ymin').text) / height)
        ymaxs.append(float(bndbox.find('ymax').text) / height)
        classes_text.append(class_name.encode('utf8'))
        classes.append(LABEL_MAP[class_name])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(filename.encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_image),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def main():
    writer = tf.io.TFRecordWriter(OUTPUT_PATH)
    xml_files = glob.glob(os.path.join(ANNOTATION_DIR, '*.xml'))
    count = 0

    for xml_file in xml_files:
        tf_example = create_tf_example(xml_file)
        if tf_example:
            writer.write(tf_example.SerializeToString())
            count += 1

    writer.close()
    print(f'✅ TFRecord 생성 완료: {count}개 예제 포함')

if __name__ == '__main__':
    main()


[⚠️] 이미지 없음: pic_1136.png, 스킵
[⚠️] 이미지 없음: pic_070.jpeg, 스킵
[⚠️] 이미지 없음: pic_286.png, 스킵
[⚠️] 이미지 없음: pic_126.jpeg, 스킵
[⚠️] 이미지 없음: pic_686.jpeg, 스킵
[⚠️] 이미지 없음: pic_1174.jpeg, 스킵
[⚠️] 이미지 없음: pic_367.png, 스킵
[⚠️] 이미지 없음: pic_921.png, 스킵
[⚠️] 이미지 없음: pic_841.png, 스킵
[⚠️] 이미지 없음: pic_201.jpeg, 스킵
[⚠️] 이미지 없음: pic_774.png, 스킵
[⚠️] 이미지 없음: pic_691.jpeg, 스킵
[⚠️] 이미지 없음: pic_792.png, 스킵
[⚠️] 이미지 없음: pic_472.png, 스킵
[⚠️] 이미지 없음: pic_961.jpeg, 스킵
[⚠️] 이미지 없음: pic_298.jpeg, 스킵
[⚠️] 이미지 없음: pic_307.png, 스킵
[⚠️] 이미지 없음: pic_075.PNG, 스킵
[⚠️] 이미지 없음: pic_911.png, 스킵
[⚠️] 이미지 없음: pic_179.jpeg, 스킵
[⚠️] 이미지 없음: pic_244.png, 스킵
[⚠️] 이미지 없음: pic_671.jpeg, 스킵
[⚠️] 이미지 없음: pic_743.png, 스킵
[⚠️] 이미지 없음: pic_192.png, 스킵
[⚠️] 이미지 없음: pic_1048.jpeg, 스킵
[⚠️] 이미지 없음: pic_095.jpeg, 스킵
[⚠️] 이미지 없음: pic_344.png, 스킵
[⚠️] 이미지 없음: pic_983.jpeg, 스킵
[⚠️] 이미지 없음: pic_857.png, 스킵
[⚠️] 이미지 없음: pic_932.png, 스킵
[⚠️] 이미지 없음: pic_265.Jpeg, 스킵
[⚠️] 이미지 없음: pic_332.jpeg, 스킵
[⚠️] 이미지 없음: pic_276.png, 스킵
[⚠️] 이미지 없음: pic_822.png,

In [12]:
!mkdir -p pretrained
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz -C pretrained

--2025-08-05 13:27:08--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.2.207, 142.250.141.207, 74.125.137.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.2.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  81.7MB/s    in 0.2s    

2025-08-05 13:27:09 (81.7 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320

In [14]:
!mkdir -p training

In [15]:
#config파일 수정
import re

def modify_pipeline_config(input_path, output_path):
    with open(input_path, 'r') as f:
        config = f.read()

    # num_classes 수정 (90 → 1)
    config = re.sub(r'num_classes:\s*\d+', 'num_classes: 1', config)

    # fine_tune_checkpoint 경로 수정
    checkpoint_path = 'pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
    config = re.sub(r'fine_tune_checkpoint:\s*".*"', f'fine_tune_checkpoint: "{checkpoint_path}"', config)

    # label_map_path 수정
    config = re.sub(r'label_map_path:\s*".*"', 'label_map_path: "label_map.pbtxt"', config)

    # train_input_reader input_path 수정
    config = re.sub(r'(train_input_reader\s*\{[^}]*input_path:\s*")[^"]*(")', r'\1train.record\2', config, flags=re.DOTALL)

    # eval_input_reader input_path 수정
    config = re.sub(r'(eval_input_reader\s*\{[^}]*input_path:\s*")[^"]*(")', r'\1val.record\2', config, flags=re.DOTALL)

    # eval_input_reader shuffle 수정 (false)
    config = re.sub(r'(eval_input_reader\s*\{[^}]*shuffle:\s*)true', r'\1false', config, flags=re.DOTALL)

    # eval_input_reader num_epochs 수정 (1)
    if 'num_epochs:' not in config:
        # eval_input_reader 블록에 num_epochs: 1 추가
        config = re.sub(r'(eval_input_reader\s*\{)', r'\1\n  num_epochs: 1', config, count=1, flags=re.DOTALL)

    # batch_size 조절 (필요하면 32로 변경)
    config = re.sub(r'batch_size:\s*\d+', 'batch_size: 32', config)

    # num_steps 조절 (필요하면 10000으로 변경)
    config = re.sub(r'num_steps:\s*\d+', 'num_steps: 10000', config)

    with open(output_path, 'w') as f:
        f.write(config)
    print(f'✅ 수정된 config 파일 저장 완료: {output_path}')


if __name__ == '__main__':
    input_config = 'pretrained/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'  # 원본 경로
    output_config = 'training/ssd_mobilenet_pipeline.config'  # 저장할 경로
    modify_pipeline_config(input_config, output_config)


✅ 수정된 config 파일 저장 완료: training/ssd_mobilenet_pipeline.config


In [17]:
# 1) 프로토콜 버퍼 설치 및 컴파일
!apt-get install -qq protobuf-compiler
!git clone --depth 1 https://github.com/tensorflow/models.git
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.

# 2) 패키지 설치
!pip install -q tensorflow_object_detection_api

# 3) 환경변수 설정 (코랩 셸용)
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim'

Cloning into 'models'...
remote: Enumerating objects: 4356, done.
remote: Counting objects: 100% (4356/4356), done.
remote: Compressing objects: 100% (3173/3173), done.
remote: Total 4356 (delta 1185), reused 3993 (delta 1110), pack-reused 0 (from 0)
Receiving objects: 100% (4356/4356), 69.97 MiB | 27.58 MiB/s, done.
Resolving deltas: 100% (1185/1185), done.
/content/models/research


In [20]:
# protobuf 컴파일
!protoc object_detection/protos/*.proto --python_out=.

# tensorflow object detection API 설치
!pip install -q tensorflow_object_detection_api

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim'

In [22]:
!python object_detection/model_main_tf2.py \
  --model_dir=training/ \
  --pipeline_config_path=training/ssd_mobilenet_pipeline.config \
  --num_train_steps=10000 \
  --sample_1_of_n_eval_examples=1 \
  --alsologtostderr


2025-08-05 13:36:41.950531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754401001.968816    9412 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754401001.975205    9412 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Traceback (most recent call last):
  File "/content/models/research/object_detection/model_main_tf2.py", line 31, in <module>
    from object_detection import model_lib_v2
  File "/content/models/research/object_detection/model_lib_v2.py", line 29, in <module>
    from object_detection import eval_util
  File "/content/models/research/object_detection/eval_util.py", line 35, in <module>
    from object_detection.metrics import coco_e

In [23]:
!pip install protobuf==3.20.3
!protoc object_detection/protos/*.proto --python_out=.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [2]:
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research


In [3]:
!python object_detection/model_main_tf2.py \
  --model_dir=training/ \
  --pipeline_config_path=training/ssd_mobilenet_pipeline.config \
  --num_train_steps=10000 \
  --sample_1_of_n_eval_examples=1 \
  --alsologtostderr


2025-08-05 13:38:57.290367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754401137.308306   10094 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754401137.313708   10094 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-05 13:38:57.332358: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/content/models/research/object_detection/model_main_tf2.py", line 31, in 

In [5]:
!pip uninstall -y tensorflow_object_detection_api
import os
os.environ['PYTHONPATH'] = '/content/models/research:/content/models/research/slim'
!protoc /content/models/research/object_detection/protos/*.proto --python_out=/content/models/research --proto_path=/content/models/research

In [6]:
!protoc /content/models/research/object_detection/protos/*.proto --python_out=/content/models/research --proto_path=/content/models/research

In [7]:
!python object_detection/model_main_tf2.py \
  --model_dir=training/ \
  --pipeline_config_path=training/ssd_mobilenet_pipeline.config \
  --num_train_steps=10000 \
  --sample_1_of_n_eval_examples=1 \
  --alsologtostderr

2025-08-05 13:40:45.774113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754401245.793974   10612 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754401245.799890   10612 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-05 13:40:45.819748: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/content/models/research/object_detection/model_main_tf2.py", line 31, in 

In [8]:
!pip install lvis

In [3]:
!python3 /content/models/research/object_detection/model_main_tf2.py \
  --model_dir=/content/training/ \
  --pipeline_config_path=/content/training/ssd_mobilenet_pipeline.config \
  --num_train_steps=10000 \
  --sample_1_of_n_eval_examples=1 \
  --alsologtostderr

2025-08-05 13:45:11.420891: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-05 13:45:11.527261: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-05 13:45:11.527898: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-05 13:45:13.729416: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/models/research/object_detection/model_main_tf2.py", line 31, in <module>
    from object_detection import model_lib_v2
ModuleNotFoundError: No module named 'object_detection'


In [4]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

In [6]:
# protobuf 컴파일
!protoc --proto_path=/content/models/research --python_out=/content/models/research /content/models/research/object_detection/protos/*.proto

In [10]:
!pip uninstall -y tensorflow
!pip install tensorflow==2.12

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58

In [7]:
# 1. 시스템 기본 패키지 업데이트 및 필수 라이브러리 설치
!apt-get update
!apt-get install -y protobuf-compiler python3-pil python3-lxml python3-tk

# 2. protobuf 버전 강제 하향 (3.20.3 권장)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

# 3. TF OD API 레포 클론 (이미 있다면 건너 뛰기)
import os
if not os.path.exists('/content/models'):
    !git clone https://github.com/tensorflow/models.git /content/models

# 4. protoc 컴파일 (protobuf 파일을 python 코드로 변환)
!cd /content/models/research && protoc object_detection/protos/*.proto --python_out=.

# 5. PYTHONPATH 환경 변수 설정 (세션 내에서 반드시 해줘야 함)
import os
os.environ['PYTHONPATH'] = "/content/models/research:/content/models/research/slim"

# 6. 필수 python 패키지 설치
!pip install tf_slim
!pip install tensorflow==2.12.0  # TF 버전은 필요에 맞게 조절 가능

# 7. 설치 검증: object_detection 모듈 임포트 테스트
import sys
sys.path.append('/content/models/research')
sys.path.append('/content/models/research/slim')
try:
    import object_detection
    print("✅ object_detection 임포트 성공!")
except Exception as e:
    print("❌ 임포트 실패:", e)


Selecting previously unselected package libimagequant0:amd64.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../0-libimagequant0_2.17.0-1_amd64.deb ...
Unpacking libimagequant0:amd64 (2.17.0-1) ...
Selecting previously unselected package libraqm0:amd64.
Preparing to unpack .../1-libraqm0_0.7.0-4ubuntu1_amd64.deb ...
Unpacking libraqm0:amd64 (0.7.0-4ubuntu1) ...
Selecting previously unselected package python3-soupsieve.
Preparing to unpack .../2-python3-soupsieve_2.3.1-1_all.deb ...
Unpacking python3-soupsieve (2.3.1-1) ...
Selecting previously unselected package python3-bs4.
Preparing to unpack .../3-python3-bs4_4.10.0-2_all.deb ...
Unpacking python3-bs4 (4.10.0-2) ...
Selecting previously unselected package python3-chardet.
Preparing to unpack .../4-python3-chardet_4.0.0-1_all.deb ...
Unpacking python3-chardet (4.0.0-1) ...
Selecting previously unselected package python3-webencodings.
Preparing to unpack .../5-python3-webencodings_0.5.1-4